<a href="https://colab.research.google.com/github/AlexeyUdod/dvp/blob/master/dvp_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Итоговая работа
=====================

Поздравляю вас, теперь вы вооружены знаниями по предобработке данных, и пора применить их на интересном кейсе.

Представлены данные такси, по которым мы должны предсказать возьмёт ли таксист заказ или нет(driver_response).




**1. Загрузите датасет taxi.csv.**




In [1]:
import platform
if platform.system() == 'Linux':
    path = '/content/drive/My Drive/Colab Notebooks/dvp/dvp.fin/'
    from google.colab import drive
    drive.mount('/content/drive')
else:
    path = ''

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#  !jupyter nbconvert --to html /content/drive/My Drive/Colab Notebooks/dvp/dvp.fin/dvp.fin.ipynb

# import os
# os.system(f'jupyter nbconvert --to html dvp.fin.ipynb')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
data = pd.read_csv(path +'taxi.csv')

**2. Посмотрите на данные. Отобразите общую информацию по признакам (вспомните о describe и info). Напишите в markdown свои наблюдения.**


In [16]:
data

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response
0,1105373,5,20,6080,174182,55.818842,37.334562,55.814567,37.355010,-1.000,-1.000,Economy,private,0
1,759733,5,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,Standard,private,1
2,416977,6,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,Economy,private,0
3,889660,2,6,6080,163522,55.745922,37.421748,55.743469,37.431130,-1.000,-1.000,Economy,private,1
4,1120055,4,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,Economy,private,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,726210,0,2,2442,27360,55.961246,37.413497,55.966694,37.415754,17.378,29.817,Economy,private,0
99996,881472,0,1,2442,381960,55.961246,37.413497,55.964892,37.413543,7.751,12.683,Economy,private,0
99997,626673,1,20,2442,627126,55.961315,37.413497,55.962203,37.416373,46.983,46.217,Standard,private,0
99998,672584,6,21,2442,140623,55.961354,37.413864,55.964892,37.413543,50.029,51.100,Economy,private,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   offer_gk                100000 non-null  int64  
 1   weekday_key             100000 non-null  int64  
 2   hour_key                100000 non-null  int64  
 3   driver_gk               100000 non-null  int64  
 4   order_gk                100000 non-null  int64  
 5   driver_latitude         100000 non-null  float64
 6   driver_longitude        100000 non-null  float64
 7   origin_order_latitude   100000 non-null  float64
 8   origin_order_longitude  100000 non-null  float64
 9   distance_km             100000 non-null  float64
 10  duration_min            100000 non-null  float64
 11  offer_class_group       100000 non-null  object 
 12  ride_type_desc          100000 non-null  object 
 13  driver_response         100000 non-null  int64  
dtypes: float64(6), int64(

In [7]:
data.describe()

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,driver_response
count,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,5.662189e+05,3.159860,13.827790,3446.376360,460867.01291,55.700824,37.492292,55.772440,37.541637,18.393518,23.134841,0.492480
std,3.259360e+05,2.054222,7.306496,2038.738677,266064.98953,2.135903,1.454640,0.665985,0.467758,41.129296,33.890105,0.499946
min,4.000000e+00,0.000000,0.000000,14.000000,18.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,2.855552e+05,1.000000,8.000000,1421.000000,229178.50000,55.708651,37.413497,55.708032,37.407117,-1.000000,-1.000000,0.000000
50%,5.656715e+05,3.000000,16.000000,3506.500000,462420.00000,55.772904,37.533664,55.772561,37.532537,10.411500,19.700000,0.000000
75%,8.487198e+05,5.000000,20.000000,5242.000000,690894.00000,55.960667,37.624113,55.962000,37.624358,30.665250,37.867000,1.000000
max,1.130355e+06,6.000000,23.000000,6806.000000,920344.00000,56.174097,38.280547,56.311121,38.243608,5234.283000,3958.283000,1.000000


In [8]:
data.boxplot()
plt.xticks(rotation=90)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 <a list of 12 Text major ticklabel objects>)

In [0]:
data_num = data[[x for x in data if x not in ['offer_class_group',	'ride_type_desc']]]
data_norm = pd.DataFrame(MinMaxScaler(feature_range=(-1,1)).fit_transform(data_num), columns=data_num.columns)
data_norm.boxplot()
plt.gcf().set_size_inches(20,10)

In [0]:
sns.boxplot(data=data_norm)
plt.gcf().set_size_inches(20,10)

In [11]:
sns.distplot(data_norm.driver_gk)
sns.distplot(data_norm.offer_gk)

данные неплохие, есть ненужные и дублирующие колонки, надо будет проверить выбросы на координатах, дистанции и времени.


**3. Выявите пропуски, а также возможные причины их возникновения. Решите, что следует сделать с ними. Напишите в markdown свои наблюдения.**


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   offer_gk                100000 non-null  int64  
 1   weekday_key             100000 non-null  int64  
 2   hour_key                100000 non-null  int64  
 3   driver_gk               100000 non-null  int64  
 4   order_gk                100000 non-null  int64  
 5   driver_latitude         100000 non-null  float64
 6   driver_longitude        100000 non-null  float64
 7   origin_order_latitude   100000 non-null  float64
 8   origin_order_longitude  100000 non-null  float64
 9   distance_km             100000 non-null  float64
 10  duration_min            100000 non-null  float64
 11  offer_class_group       100000 non-null  object 
 12  ride_type_desc          100000 non-null  object 
 13  driver_response         100000 non-null  int64  
dtypes: float64(6), int64(

In [13]:
data.isnull().sum()

offer_gk                  0
weekday_key               0
hour_key                  0
driver_gk                 0
order_gk                  0
driver_latitude           0
driver_longitude          0
origin_order_latitude     0
origin_order_longitude    0
distance_km               0
duration_min              0
offer_class_group         0
ride_type_desc            0
driver_response           0
dtype: int64

Пропусков как таковых не заметно.

In [14]:
data.drop_duplicates().shape

(100000, 14)

Дубликатов не выявлено.


**4. Оцените зависимости переменных между собой. Используйте корреляции. Будет хорошо, если воспользуетесь profile_report. Напишите в markdown свои наблюдения.**


In [15]:
# !pip install pandas_profiling
data.profile_report()

AttributeError: ignored

In [0]:
data.ProfileReport()

In [0]:
pandas_profiling.ProfileReport(data)

In [0]:
sns.scatterplot(data.groupby('driver_gk').driver_response.agg('mean').sort_values(ascending=False), 
                data.distance_km, 
                hue=data.driver_response)
plt.gcf().set_size_inches(20,10)

In [0]:
data.driver_gk.unique().shape

гипотеза что реакция водителя зависит от класса заказа

In [0]:
data.groupby('offer_class_group').driver_response.agg('mean').sort_values(ascending=False).plot()

In [0]:
drive_gk1 = data.groupby('driver_gk').driver_response.agg('mean').sort_values(ascending=False).reset_index()
sns.scatterplot(data = drive_gk1)

In [0]:
data1 = data[data.distance_km<100]
data2 = data[data.distance_km>100]
sns.jointplot(data1.duration_min, data1.driver_gk, color='Red')
plt.gcf().set_size_inches(20,10)
sns.jointplot(data2.duration_min, data2.driver_gk, color='Blue')
plt.gcf().set_size_inches(20,10)

In [0]:
sns.heatmap(data.corr(),cmap='seismic', vmax=1, vmin=-1, annot=True)
plt.gcf().set_size_inches(20,10)

In [0]:
data1 = data[(data.driver_latitude>10)&(data.driver_longitude>10)&(data.origin_order_latitude>10)&(data.origin_order_longitude>10)]
# data2 = data[data.driver_latitude>10]
sns.scatterplot( data1.driver_longitude,data1.driver_latitude, hue=data1.offer_class_group)
# sns.scatterplot(data1.origin_order_latitude, data1.origin_order_longitude, hue=data1.driver_response, palette='Blues')
plt.gcf().set_size_inches(20,20)

В пределах МКАДа больше всего дорогих классов такси.

Визуально похоже, что на ответ водителя больше всего зависит от  конкретного водителя и от класса заказа.

**5. Определите стратегию преобразования категориальных признаков (т.е. как их сделать адекватными для моделей).**


Так как мы применяем random forest то будем использовать label encoder

In [0]:
data['offer_class_group'] = LabelEncoder().fit_transform(data['offer_class_group'])
data['ride_type_desc'] = LabelEncoder().fit_transform(data['ride_type_desc'])
data

**6. Найдите признаки, которые можно разделить на другие, или преобразовать в другой тип данных. Удалите лишние, при необходимости.**


удалим ненужные колонки offer_gk, order_gk, distance_km, так как номера заказа и предложения не имеют значения для целевой метрики, а дистанция будет дублировать более важный параметр duration_min, по которому водители и будут ориентироваться.

In [0]:
data_d = data.drop(data.columns[[0,4,9]],axis=1)


добавим новый признак как норму разницы координат между заказом и водителем. (возможно это дублирует параметр который мы только что удалили, но это не точно, поэтому можно попробовать)

In [0]:
data_d['dist2'] = ((data_d.driver_latitude - data_d.origin_order_latitude)**2 + 
                   (data_d.driver_longitude - data_d.origin_order_longitude)**2)**0.5

In [0]:
sns.heatmap(data_d.corr(), cmap='seismic', vmax=1, vmin=-1)
plt.gcf().set_size_inches(15,9)

In [0]:
sns.pairplot(data_d, diag_kind='kde', markers='.')
plt.gcf().set_size_inches(30,30)

In [0]:
# координаты водителя - координаты заказа и проверить зависимость с дистанцией заказа

In [0]:
RF1(data, 13, range(13))

7. Разделите выборку на обучаемую и тестовую.


In [0]:
x_train, x_test, y_train, y_test = train_test_split(data.drop('driver_response', axis=1), 
                                                    data.driver_response, test_size=0.6, 
                                                    random_state=11)

In [0]:
x_train

8. Обучите модель. Напишите в markdown свои наблюдения по полученным результатам. Хорошие результаты дают классификаторы RandomForest и XGBoost

In [0]:



x_train, x_test, y_train, y_test = train_test_split(data.drop('driver_response', axis=1), 
                                                    data.driver_response, test_size=0.6, 
                                                    random_state=11)
model = RandomForestClassifier(n_estimators=100)
model.fit(x_train, y_train)
print(model.score(x_test, y_test))
print(classification_report(y_test, model.predict(x_test)))
sns.barplot(data.drop('driver_response',axis=1).columns, model.feature_importances_, )
plt.xticks(rotation=90)
plt.gcf().set_size_inches(15,10)

In [0]:
data[data.columns[[7,3,13]]]

In [0]:
def RF1(dataset, target_column_num, input_nums):
  x_train, x_test, y_train, y_test = train_test_split(dataset.drop(dataset.columns[target_column_num], axis=1)[dataset.columns[[input_nums]]], dataset[dataset.columns[target_column_num]], test_size=0.5)
  model = RandomForestClassifier(n_estimators=100)
  model.fit(x_train, y_train)
  score =  model.score(x_test, y_test)
  real_score = round(100*(score - np.float(1 / dataset[dataset.columns[target_column_num]].unique().shape[0])), 3) # реальные очки модели с вычетом случайного угадывания
  # print('model score', score, type(score))
  # print('real score', real_score,'%') 
  # print(classification_report(y_test, model.predict(x_test)))
  # sns.barplot(dataset.drop(dataset.columns[target_column_num], axis=1)[dataset.columns[[input_nums]]].columns, model.feature_importances_)
  # plt.xticks(rotation=90)
  # plt.gcf().set_size_inches(15,8)
  return real_score

In [0]:
data['driver_response'].unique().shape[0]

In [0]:
type(s)

In [0]:
data.columns

In [0]:
s = RF1(data, 13, range(13))
s

In [0]:
data.info()

In [0]:
data_t2 = data[(data.driver_latitude > 20) & (data.origin_order_latitude > 20)]
RF1(data_t2, 13, range(13)[5:8])

In [0]:
results = pd
for q in range(0, 45, 5):
  data_t1 = data[(data.driver_latitude < data.driver_latitude.quantile(1 - q/100)) & (data.driver_gk > data.driver_gk.quantile(q/100))]
  results = RF1(data_t1, 13, range(13))
  print(q, results)

  
#   results = pd.DataFrame(data={'Feature':data.drop('driver_response',axis=1).columns,'Importance':model.feature_importances_}).sort_values(by='Importance', ascending=False)
# sns.barplot(results.Feature, results.Importance)
# plt.xticks(rotation=90)
# plt.gcf().set_size_inches(15,10)

In [0]:
results = pd.DataFrame(data={'Feature':data.drop('driver_response',axis=1).columns,'Importance':model.feature_importances_}).sort_values(by='Importance', ascending=False)
sns.barplot(results.Feature, results.Importance)
plt.xticks(rotation=90)
plt.gcf().set_size_inches(15,10)

In [0]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix('demo/data/agaricus.txt.train')
dtest = xgb.DMatrix('demo/data/agaricus.txt.test')
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest) 

In [0]:
!jupyter nbconvert --to html hw14-2.ipynb